## Imports

In [1]:
from src.db.db_connection import DBConnection
from src.backend.ragtag import RagTag

## Setup and methods def

In [2]:
class RagTagIntegration():
    def __init__(self):
        self.db_path = "test_database.db"
        self.db_connection = DBConnection(self.db_path)
        
        self.conn = self.db_connection.create_connection()
        self.setupDatabaseSchema()
        self.ragtag = RagTag(self.db_connection)

    def setupDatabaseSchema(self):
        schema_files = [
            'src/db/schema/create_docs_table.sql',
            'src/db/schema/create_tags_table.sql',
            'src/db/schema/create_doc_tags_table.sql'
        ]
        cursor = self.conn.cursor()
        for script_path in schema_files:
            with open(script_path, 'r') as sql_file:
                sql_script = sql_file.read()
            cursor.executescript(sql_script)
        self.conn.commit()
    
    def tearDownDB(self):
        cursor = self.conn.cursor()
        cursor.execute("DROP TABLE docs")
        cursor.execute("DROP TABLE tags")
        cursor.execute("DROP TABLE doc_tags")
        self.conn.commit()

        cursor.close()
        self.conn.close()

    def add_test_documents(self):
        documents = [
            "Python is a programming language with a simple syntax.",
            "To create a list in Python, use square brackets.",
            "Pancakes usually contain flour, eggs, and milk."
        ]
        return self.ragtag.add_documents_bulk(documents)


## Instantiate RAG Engine
### Add test documents

In [3]:
rageng = RagTagIntegration()
rageng.add_test_documents()

[1, 2, 3]

## Run a search

In [4]:
# Search for documents containing the word "Python"
search_results = rageng.ragtag.search_documents("Python")
print(f"Found {len(search_results)} documents matching 'Python':\n\n{search_results}")
print([rageng.ragtag.document_dao.read_document(doc[0]) for doc in search_results])

# Search for documents containing the word "Pancakes"
search_results = rageng.ragtag.search_documents("Pancakes")
print(f"Found {len(search_results)}  documents matching 'Pancakes':\n\n{search_results}")
print([rageng.ragtag.document_dao.read_document(doc[0]) for doc in search_results])


Found 2 documents matching 'Python':

[(1, 0.951409296927534), (2, 0.951409296927534)]
['Python is a programming language with a simple syntax.', 'To create a list in Python, use square brackets.']
Found 1  documents matching 'Pancakes':

[(3, 1.7301927164785618)]
['Pancakes usually contain flour, eggs, and milk.']


## Cleanup

In [5]:
rageng.tearDownDB()